In [77]:
from scipy.spatial.transform import Rotation as R
import numpy as np

rot = R.from_rotvec(np.pi/2*np.array([1,0,0]))
rot_mat = rot.as_matrix()
# print(rot_mat)
# print(type(rot_mat))

v = np.array([[0],[1],[0]])
v = np.transpose(v)

# print(v.shape)
# print(v)

v_after = np.matmul(v,rot_mat)

# print(v_after)

translation = [3,4,5]

transform = np.eye(4)
transform[0:3,0:3] = rot.as_matrix()
transform[0:3,3] = translation

print(transform)


[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  3.00000000e+00]
 [ 0.00000000e+00  2.22044605e-16 -1.00000000e+00  4.00000000e+00]
 [ 0.00000000e+00  1.00000000e+00  2.22044605e-16  5.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


array([[3.],
       [4.],
       [6.],
       [1.]])

In [42]:
from scipy.spatial.transform import Rotation as R
import numpy as np

rot = R.from_rotvec(np.pi/2*np.array([1,0,0]))
rot_mat = rot.as_matrix()
print(rot_mat)
print(type(rot_mat))

v = np.array([[0],[1],[0]])
print(v.shape)
print(v)

v_after = np.matmul(rot_mat,v)

print(v_after)

[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  2.22044605e-16 -1.00000000e+00]
 [ 0.00000000e+00  1.00000000e+00  2.22044605e-16]]
<class 'numpy.ndarray'>
(3, 1)
[[0]
 [1]
 [0]]
[[0.00000000e+00]
 [2.22044605e-16]
 [1.00000000e+00]]


In [1]:
import tensorflow as tf

def append_coulmn_to_tensor(y_source_net_selected):
    batch_size = tf.shape(y_source_net_selected)[0]
    point_count = tf.shape(y_source_net_selected)[1]
    ones = tf.ones([batch_size, point_count, 1])

    print(y_source_net_selected.shape)

    return tf.concat([y_source_net_selected, ones], -1)

def calculate_target_using_transformation(transform, y_source):
    return tf.transpose(tf.linalg.matmul(transform, y_source, transpose_b=True),[0,2,1])[...,0:3]

def create_result_transform_matrix(R, T):
    lastRow = tf.reshape(tf.constant(np.array([0,0,0,1]), tf.float32), (1,1,4))
    lastRow = tf.repeat(lastRow,tf.shape(R)[0],0)
    return tf.concat([tf.concat([R, tf.transpose(T,[0,2,1])], axis = -1), lastRow], axis = -2)

def calculate_transform_pred(y_source_net_selected, y_target_net_calculated):
    mean_d = tf.math.reduce_mean(y_target_net_calculated, axis = 1, keepdims = True)
    mean_m = tf.math.reduce_mean(y_source_net_selected, axis = 1, keepdims = True)
    diff_d = tf.math.subtract(y_target_net_calculated, mean_d)
    diff_m = tf.math.subtract(y_source_net_selected, mean_m)

    ## diff_m ^ T * diff_d
    H = tf.linalg.matmul(tf.transpose(diff_m, [0, 2, 1]), diff_d)

    # SVD
    S, U, V = tf.linalg.svd(H, full_matrices = True, compute_uv = True)

    ## V * U^T
    R = tf.linalg.matmul(V, tf.transpose(U, [0, 2, 1]))

    T = tf.math.subtract(mean_d, mean_m)

    return create_result_transform_matrix(R, T)

def calculate_distance(y1, y2):
    return tf.math.sqrt(tf.math.reduce_sum((y1 - y2)**2, axis=-1))

 

In [2]:
import tensorflow as tf
from scipy.spatial.transform import Rotation as R
import numpy as np

def batch(smth):
    shape=list(smth.shape)
    shape.insert(0,1)
    batched = np.zeros(shape,dtype='float32')
    batched[0] = smth
    return batched

def RT(rot,t):
    rt = np.zeros((4,4),dtype='float32')
    rt[0:3,0:3] = rot
    rt[0:3,3] = t
    
    return rt

def append1(smth):
    shape=list(smth.shape)
    shape[-1] = shape[-1]+1
    
    res = np.ones(shape,dtype='float32')
    res[...,0:-1] = smth
    return res

    

    
    
    

rot_axis = np.array([0,1,7])
rot_axis = rot_axis/np.linalg.norm(rot_axis)
# print(rot_axis)

x = [[0,1,-6],[0,-1,6],[-3,2,3],[3,-2,-3],[-12,6,2],[12,-6,-2]]
x = np.array(x ,dtype='float32')
x = np.random.random((30,3)).astype('float32')*100
x = np.concatenate([x,-x])  #make pioints symmetrical so there is no transform after rotation
x_b = batch(x)

t = [0,5,0]
t = np.array(t ,dtype='float32')

rot = R.from_rotvec(np.pi/2*rot_axis)
rot = rot.as_matrix()

transform = RT(rot,t)
print("GT transform")
print(transform)
transform_b = batch(transform)


x_t = np.transpose(x)
# print("transposed source")
# print(x_t)
x_target = np.matmul(rot,x_t)
print("GT target")
print(np.transpose(x_target)[0:10]+t)

print("=================")
print("Pred transform")

res = calculate_transform_pred(tf.Variable(x_b.astype('float32')), tf.Variable(batch(np.transpose(x_target)+t).astype('float32')))
padded1 = append1(x_b)
transformed = calculate_target_using_transformation(res,padded1)


print(res.numpy())
print("Pred target")
print(transformed.numpy()[0][0:10])

# res = calculate_transform_pred()










GT transform
[[ 1.6653345e-16 -9.8994946e-01  1.4142136e-01  0.0000000e+00]
 [ 9.8994946e-01  2.0000000e-02  1.4000000e-01  5.0000000e+00]
 [-1.4142136e-01  1.4000000e-01  9.8000002e-01  0.0000000e+00]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00]]
GT target
[[-38.77639808  17.10018904  71.768637  ]
 [-89.61134588  58.73315256  67.97583687]
 [ -6.39015897  64.84824175  81.65320097]
 [-62.31856704  15.25641711  26.82925295]
 [-16.41535699 110.06130153  86.23282769]
 [-53.04396164  93.92813026  32.94566194]
 [-83.24216862  32.55415182  69.78635266]
 [-93.00848909  12.96959827  21.66046654]
 [-55.69796744  22.4596448   57.18540512]
 [  1.22930556  63.19661273  52.52685256]]
Pred transform
(1, 3, 3)
[[[-2.0861626e-07 -9.8994946e-01  1.4142144e-01  2.0345053e-06]
  [ 9.8994946e-01  1.9999772e-02  1.4000025e-01  5.0000005e+00]
  [-1.4142159e-01  1.4000013e-01  9.8000002e-01  1.0172527e-06]
  [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  1.0000000e+00]]]
Pred target
[[-38.77